## Imports

In [ ]:
import pandas as pd
import re, gensim, sys, json
from gensim.models import Word2Vec
import tensorflow as tf
from sklearn.utils import shuffle
import numpy as np
from gensim.models import Word2Vec, Sequential
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.utils import to_categorical

from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.optimizers import Adamax

## Parameters

In [4]:
#parameters
max_features = 5000
maxlen = 50
batch_size = 50
embedding_dims = 50
kernel_size = 3
hidden_dims = 250
epochs = 2

## Helper Functions

In [ ]:
def clean140(data):
        data = pd.read_csv(data, encoding = "latin1")
        data.columns=['sentiment', 'id', 'date', 'q', 'user', 'tweet']
        data = data.drop(['id', 'date', 'q', 'user'], axis=1)
        data = shuffle(data)
        text = data.tweet
        sentiment = data.sentiment
        sentiment = pd.Series.tolist(sentiment)
        for i in range(len(sentiment)):
            j = sentiment[i]
            if j==4:
                sentiment[i]=1
            elif j==2:
                sentiment[i]=1

        sentiment=np.asarray(sentiment)

        trainData = []
        for x in text:
            trainData.append(''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x.lower())).split())
        
        return (trainData, sentiment)
def cleanData(csvLocation, name="default"):
    if name == "140":
        data, sentiment = clean140(csvLocation)
        return (data, sentiment)
        

    data = pd.read_csv(csvLocation, sep='\t')
    data.columns = ['text', 'sentiment']
    trainData=[]
    for x in data.text:
        trainData.append(''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x.lower())).split())
        
    return (trainData, data.sentiment)

## Data and word-vectors

In [5]:
#Cleaning Data
sentence, sentiment  = cleanData("sentiment140train.csv", name= "140")

In [6]:
if os.path.exists('trainedW2V'):
    wordModel=gensim.models.Word2Vec.load("trainedW2V")
else:
    wordModel=gensim.models.Word2Vec(sentence, min_count=1, size=300, max_vocab_size=50000, iter=50, workers=50)
    wordModel.save('trainedW2V')

In [6]:
pretrained_weights = wordModel.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape

def word2idx(word):
    return wordModel.wv.vocab[word].index
def idx2word(idx):
    return wordModel.wv.index2word[idx]

/home/abd/testEnv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [7]:
pretrained_weights = wordModel.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape

/home/abd/testEnv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [8]:
def prepareData(sentence) :
    """
    This function converts words to embedding metrices.
    """
    preparedData = np.zeros([len(sentence), maxlen], dtype=np.int32)
    for i, sentences in enumerate(sentence):
      for t, word in enumerate(sentences[:-1]):
        try:
            preparedData[i, t] = word2idx(word)
        except:
            print(word + " not found!")
            pass


    print(preparedData.shape)
    return preparedData

In [9]:
trainData = np.zeros([len(sentence), maxlen], dtype=np.int32)
for i, sentences in enumerate(sentence):
  for t, word in enumerate(sentences[:-1]):
    try:
        trainData[i, t] = word2idx(word)
    except:
        print(word + " not found!")
        pass
            

print(trainData.shape)

## Model

In [11]:
with tf.device('/gpu:2'):
    print('Build model...')
    model = Sequential()

    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    model.add(Embedding(max_features,
                        embedding_dims,
                        input_length=maxlen))

    model.add(Dropout(0.5))

    # we add a Convolution1D, which will learn filters
    # word group filters of size filter_length:
    model.add(Conv1D(filters,
                     kernel_size,
                     padding='valid',
                     activation='relu',
                     strides=1))
    # we use max pooling:
    model.add(GlobalMaxPooling1D())

    # We add a vanilla hidden layer:
    model.add(Dense(hidden_dims))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))

    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer=Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0),
                  metrics=['accuracy'])
    model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            250000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 48, 250)           37750     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropou

## Fitting

In [15]:
model.fit(trainData, sentiment, validation_split=0.25, epochs=10, batch_size=100)

Train on 1199999 samples, validate on 400000 samples
Epoch 1/10
1199999/1199999 [==============================] - 96s 80us/step - loss: 0.5249 - acc: 0.7331 - val_loss: 0.5245 - val_acc: 0.7330
Epoch 2/10
1199999/1199999 [==============================] - 92s 76us/step - loss: 0.5296 - acc: 0.7311 - val_loss: 0.5092 - val_acc: 0.7479
Epoch 3/10
1199999/1199999 [==============================] - 93s 78us/step - loss: 0.5182 - acc: 0.7415 - val_loss: 0.5007 - val_acc: 0.7541
Epoch 4/10
1199999/1199999 [==============================] - 95s 79us/step - loss: 0.5099 - acc: 0.7480 - val_loss: 0.4925 - val_acc: 0.7598
Epoch 5/10
1199999/1199999 [==============================] - 92s 77us/step - loss: 0.5050 - acc: 0.7520 - val_loss: 0.4859 - val_acc: 0.7646
Epoch 6/10
1199999/1199999 [==============================] - 92s 77us/step - loss: 0.4998 - acc: 0.7562 - val_loss: 0.4867 - val_acc: 0.7641
Epoch 7/10
1199999/1199999 [==============================] - 94s 79us/step - loss: 0.4969 - ac

In [9]:
# model.save('project140.h5')
from keras.models import load_model
model=load_model('project140.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


## Inference

In [10]:
def predictSentiment(x):
    x=[x]
    a=[]
    for i in x:
        a.append(''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",i.lower())).split())
    trainData = np.zeros([len(a), maxlen], dtype=np.int32)
    for i, sentences in enumerate(a):
      for t, word in enumerate(sentences[:]):
        try:
            trainData[i, t] = word2idx(word)
        except:
            print(word + " not found!")
            pass
    
    score = model.predict(trainData)
    print(score)
    print("Positive with ", score*100, "% probability.")


In [11]:
predictSentiment('')

[[0.58085936]]
Positive with  [[58.085938]] % probability.


## Sentiment140 testData inference and evaluation

In [12]:
test, testSent = cleanData("sentiment140test.csv", name= "140")

In [16]:
testData=prepareData(test)

In [17]:
predictions = model.predict(testData)

In [18]:
errors = testSent-predictions.round().flatten()

In [19]:
len(errors[errors==0])/len(errors)

0.6438631790744467

# CoreML

In [24]:
import coremltools

In [25]:
outputLabels = ['Positive', 'Negative']

In [ ]:
coreMLSentiment = coremltools.converters.keras.convert('project140.h5', input_names=['sentence'], output_names=['sentiment'], class_labels=outputLabels)

In [224]:
print(coreMLSentiment)

input {
  name: "sentence"
  type {
    multiArrayType {
      shape: 1
      dataType: DOUBLE
    }
  }
}
output {
  name: "sentiment"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "classLabel"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "sentiment"



In [227]:
coreMLSentiment.author = 'iabd.me'
coreMLSentiment.license = 'Dusk'
coreMLSentiment.short_description = 'Sentiment prediction (Sentiment140)'
coreMLSentiment.input_description['sentence'] = 'Input sentence'
coreMLSentiment.output_description['sentiment'] = 'Probability of each sentiment'
coreMLSentiment.output_description['classLabel'] = 'Labels of sentiment'

In [228]:
coreMLSentiment.save('sentimentClassifier.mlmodel')